This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'B6Ye5coT7-ZzdxT2My4K'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests

In [23]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json"
params = dict([('key',API_KEY),('start_date', '2017-01-01'),('end_date', '2017-12-31')])
response = requests.get(url,params)

In [25]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(response.headers)


{'Date': 'Fri, 17 Jan 2020 11:20:49 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '6663', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=ddc9146afd0e172f44aa8dc4711098a4c1579260047; expires=Sun, 16-Feb-20 11:20:47 GMT; path=/; domain=.quandl.com; HttpOnly; SameSite=Lax', 'Allow': 'GET, HEAD, POST, PUT, DELETE, OPTIONS, PATCH', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Content-Encoding': 'gzip', 'ETag': 'W/"2584c5d247ed4d86439dd1110bba168f"', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Rack-CORS': 'miss; no-origin', 'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '48', 'X-Request-Id': '12c91800-aeac-470c-b23e-6b7b46507249', 'X-Runtime': '0.094230', 'X-XSS-Protection': '1; mode=block', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '5567f12389b1e346-CCU'}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Mini Project

### Solutions


In [26]:
json_data = response.json()

In [27]:
json_to_dict = dict(json_data)

In [32]:
json_to_dict.keys()

dict_keys(['dataset'])

So dataset is the only key. Will have to drill down further

In [33]:
dataset_dict = dict(json_to_dict["dataset"])

In [75]:
dataset_dict.keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [47]:
dataset_dict["column_names"]

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [66]:
high_ind = dataset_dict["column_names"].index('High')
low_ind = dataset_dict["column_names"].index('Low')
open_ind = dataset_dict["column_names"].index('Open')
close_ind = dataset_dict["column_names"].index('Close')
trad_vol = dataset_dict["column_names"].index('Traded Volume')
close_ind = dataset_dict["column_names"].index('Close')

In [77]:
max_open = 0
min_open = dataset_dict["data"][0][open_ind]
daily_change = dataset_dict["data"][0][high_ind]-dataset_dict["data"][0][low_ind]
trading_vol = 0
max_close_diff = dataset_dict["data"][0][close_ind]-dataset_dict["data"][1][close_ind]
for i in range(len(dataset_dict["data"])):
    if dataset_dict["data"][i][open_ind] is not None and dataset_dict["data"][i][open_ind]>max_open:
        max_open = dataset_dict["data"][i][open_ind]
    if dataset_dict["data"][i][open_ind] is not None and dataset_dict["data"][i][open_ind]<min_open:
        min_open = dataset_dict["data"][i][open_ind]
    if daily_change>dataset_dict["data"][i][high_ind]-dataset_dict["data"][i][low_ind]:
        daily_change = dataset_dict["data"][i][high_ind]-dataset_dict["data"][i][low_ind]
    trading_vol += dataset_dict["data"][i][trad_vol]
    
    if i!=0:
        if max_close_diff < dataset_dict["data"][i-1][close_ind]-dataset_dict["data"][i][close_ind]:
            max_close_diff - dataset_dict["data"][i-1][close_ind]-dataset_dict["data"][i][close_ind]
    
        
    
print("Highest Opening price is : {}".format(max_open))
print("Lowest Opening price is : {}".format(min_open))
print("Largest change on any day is : {}".format(daily_change))
print("Largest change between 2 days is : {}".format(max_close_diff))
print("Average Trading Volume is : {}".format(trading_vol/len(dataset_dict['data'])))


Highest Opening price is : 53.11
Lowest Opening price is : 34.0
Largest change on any day is : 0.18999999999999773
Largest change between 2 days is : 0.1599999999999966
Average Trading Volume is : 89124.33725490196


### Median trading volume

In [81]:
trading_volume = [dataset_dict['data'][i][trad_vol] for i in range(len(dataset_dict['data']))]

In [85]:
def median_volume(trade_list):
    sorted_trade = sorted(trade_list)
    median = sorted_trade[int(len(sorted_trade)/2)]
    return median

In [86]:
median = median_volume(trading_volume)

In [88]:
print("Median Trading Volume is : {}".format(median))

Median Trading Volume is : 76286.0
